In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import svm
#from keras.models import Sequential

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "c:\users\johns\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "c:\users\johns\appdata\local\programs\python\python39\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\johns\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\johns\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_ten

In [ ]:
dataset = pd.read_csv('./District/NumberingDistrict.csv')
dataset.head()

In [ ]:
# Delete outliers
dataset = dataset[dataset['unit_ntd'] > 0]
# delete = dataset[((dataset.unit_ntd > 3000000) |
#                   (dataset.total_ntd > 900000000) |
#                   (dataset.transaction_year < 2019) |
#                   (dataset.land_shift_area <= 0)
#                  )].index
delete = dataset[((dataset.unit_ntd > 600000) |
               (dataset.total_ntd > 900000000) |
               (dataset.transaction_year < 2019) | 
               (dataset.land_shift_area <= 0)
              )].index
dataset = dataset.drop(delete)
INTERVAL = 10000
dataset['unit_ntd'] = (dataset['unit_ntd'] // INTERVAL) * INTERVAL
dataset.head()

In [ ]:
X = dataset[['building_shift_total_area']]
Y = dataset[['total_ntd']]
# Normalize
minMaxScalerX = MinMaxScaler()
minMaxScalerY = MinMaxScaler()
X_scale = minMaxScalerX.fit_transform(X)
Y_scale = minMaxScalerY.fit_transform(Y)
print("X_scale : ", X_scale)
print("Y_scale : ", Y_scale)
# Split train & test set
X_train, X_test, y_train, y_test = train_test_split(X_scale, Y_scale, test_size = 0.2, random_state = 0)

In [ ]:
# Sequential train model
NUM_EPOCHS = 100
BATCH_SIZE = 100
model = Sequential()
model.add(Dense(100, input_dim = 1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# Fit model
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs = NUM_EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1
)

In [17]:
# Predict test set
y_test_predict = model.predict(X_test)
r2 = r2_score(y_test, y_test_predict)
mse = mean_squared_error(y_test, y_test_predict)
print('MSE: %.3f, R^2: %.3f' % (mse, r2))

# test = [[9, 15.8, 36, 41], [10, 7.55, 46, 41]]
# t_test = minMaxScalerX.transform(np.array(test))
# predict = model.predict(t_test)
# print(minMaxScalerY.inverse_transform(predict))

MSE: 0.000, R^2: -0.099
